In [ ]:
print((H.num_nodes, H.num_edges))

vals, counts = np.unique(
    [len(c) for c in xgi.connected_components(H)], return_counts=True
)
print(np.array([vals, counts]))

plt.figure(figsize=(8, 4))
plt.subplot(121)

h = H.nodes.degree.ashist(density=True, log_binning=True)

plt.loglog(h["bin_center"], h["value"], "ko", markersize=2)
plt.title("Degree distribution")
plt.xlabel(r"$k$", fontsize=16)
plt.ylabel(r"$P(k)$", fontsize=16)
plt.ylim([1e-5, 1])
sns.despine()

plt.subplot(122)
h = H.edges.size.ashist(density=True, log_binning=True)
plt.loglog(h["bin_center"], h["value"], "ko", markersize=2)
plt.title("Edge size distribution")
plt.xlabel(r"$s$", fontsize=16)
plt.ylabel(r"$P(s)$", fontsize=16)
plt.ylim([1e-5, 1])
sns.despine()
plt.tight_layout()
plt.savefig(f"{datasheet_folder}/{dataset_name}/stats.png", dpi=300)
plt.show()